In [1]:
import pandas as pd
from datasets import load_dataset

# Load the dataset
dataset = load_dataset("ganga4364/garchen_rinpoche_benchmark")

test_df = dataset["train"].to_pandas()
test_df.to_csv('test.csv', index=False)

README.md: 0.00B [00:00, ?B/s]

Repo card metadata block was not found. Setting CardData to empty.


benchmark_segments.csv: 0.00B [00:00, ?B/s]

full_audio_strata.csv: 0.00B [00:00, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

DatasetGenerationCastError: An error occurred while generating the dataset

All the data files must have the same columns, but at some point there are 4 new columns ({'ID', 'split_duration_seconds', 'Filename', 'AGE'}) and 6 missing columns ({'original_id', 'url', 'inference_transcript', 'audio_duration', 'age_group', 'file_name'}).

This happened while the csv dataset builder was generating data using

hf://datasets/ganga4364/garchen_rinpoche_benchmark/full_audio_strata.csv (at revision c0c0be87d61c9f0ec289dfd617898d6e444e0941)

Please either edit the data files to have matching columns, or separate them into different configurations (see docs at https://hf.co/docs/hub/datasets-manual-configuration#multiple-configurations)

In [ ]:
import os
import pandas as pd
from tqdm import tqdm
from transformers import pipeline
from datasets import Dataset, Audio
import logging
import requests

# Configure logging
logging.basicConfig(filename='failed_downloads.log', level=logging.ERROR, format='%(asctime)s - %(message)s')

# Initialize generator pipelines
generator1 = pipeline(task="automatic-speech-recognition", model="ganga4364/Garchen_Rinpoche-whisper_latin_tibetan_added_on_uni_Checkpoint-4000", device=0)
generator2 = pipeline(task="automatic-speech-recognition", model="ganga4364/Garchen_Rinpoche-wav2vec2-Checkpoint-19000", device=0)

# Function to download and validate audio
def download_audio(row):
    file_name = os.path.basename(row["url"])
    save_path = f"./downloads/{file_name}"
    os.makedirs(os.path.dirname(save_path), exist_ok=True)

    if os.path.exists(save_path):  # Skip if file exists
        return save_path

    try:
        response = requests.get(row["url"], timeout=10)
        response.raise_for_status()  # Check for HTTP errors
        with open(save_path, 'wb') as f:
            f.write(response.content)
        return save_path

    except Exception as e:
        logging.error(f"Failed to download {row['file_name']}: {e}")
        return None  # Return None if download failed

# Function to process inference in batches
def process_inference(batch):
    # Perform inference with generator1
    results1 = generator1(batch["path"])
    batch["ganga4364/Garchen_Rinpoche-whisper_latin_tibetan_added_on_uni_Checkpoint-4000_transcript"] = [result["text"] for result in results1]

    # Perform inference with generator2
    results2 = generator2(batch["path"])
    batch["ganga4364/Garchen_Rinpoche-whisper_latin_tibetan_added_on_uni_Checkpoint-4000_transcript"] = [result["text"] for result in results2]

    return batch


In [2]:
#path to test file
input_file = "benchmark_segments.csv"
output_dir = "chunks_test"
output_file = "processed_test.csv"

os.makedirs(output_dir, exist_ok=True)

In [3]:

for i, chunk in enumerate(pd.read_csv(input_file, chunksize=1000), start=1):
    if chunk.empty:
        print(f"Chunk {i} is empty. Skipping.")
        continue

    chunk_file = os.path.join(output_dir, f"chunk_{i}.csv")
    if os.path.exists(chunk_file):
        print(f"Skipping chunk {i}, already processed.")
        continue

    tqdm.pandas(desc="Downloading audio files")
    chunk["path"] = chunk.progress_apply(download_audio, axis=1)
    chunk = chunk[chunk["path"].notnull()]  # Remove rows with failed downloads

    # Reset index to avoid duplicate field errors
    chunk.reset_index(drop=True, inplace=True)

    # Convert to Dataset
    dataset = Dataset.from_pandas(chunk)
    #dataset = dataset.cast_column("path", Audio())

    # Perform batched inference
    dataset = dataset.map(process_inference, batched=True, batch_size=8)

    # Save processed chunk to CSV
    dataset.to_pandas().to_csv(chunk_file, index=False)
    print(f"Saved chunk {i} to {chunk_file}")


Parameter 'function'=<function process_inference at 0x7f06b0250af0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


Map:   0%|          | 0/978 [00:00<?, ? examples/s]

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Saved chunk 1 to chunks_test/chunk_1.csv


In [4]:
# Merge all chunk files into final output
all_chunks = [pd.read_csv(os.path.join(output_dir, f)) for f in sorted(os.listdir(output_dir)) if f.endswith(".csv")]
final_df = pd.concat(all_chunks, ignore_index=True)
final_df.to_csv(output_file, index=False)
print(f"All chunks merged and saved to {output_file}")


All chunks merged and saved to processed_test.csv


In [6]:
df = pd.read_csv("processed_test.csv")
df2 = pd.read_csv("transcription_data_latest.csv")

In [7]:
df.head()

,file_name,url,inference_transcript,audio_duration,original_id,strata,age_group,duration_category,content_type,path,inference_checkpoint-10000,inference_checkpoint-19000
0,STT_GR_0001_0010_93600_to_106200,https://d38pmlk0v88drf.cloudfront.net/wav16k/S...,ཁོང་ཨེ་ནས་དགེ་སྡུན་ཁྲད་ི་ལྔ་རྡག་རེད།་བུ་གླིང་ྡ...,12.600,STT_GR_0001,70-80__short__Teaching,70-80,short,Teaching,./downloads/STT_GR_0001_0010_93600_to_106200.wav,ཁོང་ དེ་ནས་དགེ་འདུན་ཁྲིཡི་རྔ་ཟདག་རེད། བུ་གླིང...,ཁོང་། དེ་ནས་དགེ་འདུ་ཁྲིཡི་རྔ་བདག་རེད། བུ་གླིང...
1,STT_GR_0033_0105_224320_to_225856,https://d38pmlk0v88drf.cloudfront.net/wav16k/S...,ཨ་ཕ་པ་རེ་འོ་ར་དེའ་ཐག་ནས་,1.536,STT_GR_0033,70-80__short__Teaching,70-80,short,Teaching,./downloads/STT_GR_0033_0105_224320_to_225856.wav,ཨ། ཕན་པ་རེ་འོད་ན་དེ་མཚས་ན་,ཨ། ཕན་པ་རེ་འོད་ན་དེ་ཐེང་ལུ
2,STT_GR_0032_0004_19067_to_24988,https://d38pmlk0v88drf.cloudfront.net/wav16k/S...,རུན་པོ་དེ་གས་ན་ཡ་བ་སྐུར་དེ་ལ་གསུམ་དེ་ཁོང་ཁྲོ་ད...,5.921,STT_GR_0032,70-80__short__Teaching,70-80,short,Teaching,./downloads/STT_GR_0032_0004_19067_to_24988.wav,པུ་སྟེང་ས་ལང་ཡར་སཀུན་ཟླ་ལ་གསམའདེ་ཁོན་འགྲོ་ལྔའ་...,པོ་ྟེ་བས་དི་ཡར་བ་ཀུ་་འདི་ལ་སམ་འདི་སྔོན་འགྲོ་ལྔ...
3,STT_GR_0033_0099_212224_to_213024,https://d38pmlk0v88drf.cloudfront.net/wav16k/S...,ཐར་བང་ཟར་དུས།,0.800,STT_GR_0033,70-80__short__Teaching,70-80,short,Teaching,./downloads/STT_GR_0033_0099_212224_to_213024.wav,ཐར་མ་ང་འྒྱེདས་,ཐག་མ་ཐང་ཅིད
4,STT_GR_0033_0025_56251_to_58684,https://d38pmlk0v88drf.cloudfront.net/wav16k/S...,ཁོ་དེ་གས་གོ་ཐུབ་ཀི་འདུག ཡག་པོ་འདུག་བསམ་གི་འདུག,2.433,STT_GR_0033,70-80__short__Teaching,70-80,short,Teaching,./downloads/STT_GR_0033_0025_56251_to_58684.wav,ཁོ་ཚོས་གཅིག་གི་གོ་ཐུབ་ཀྱི་འདུག ཡག་པོ་འདུག་བསམ་...,ཁོ་ཚོས་གཅིག་གི་གོ་ཐུབ་ཀྱི་འདུག ཡག་པོ་འདུག་བསམ་...


In [9]:
df2.head()

,file_name,state,inference_transcript,transcript,url,created_at,submitted_at,reviewed_at,audio_duration,original_id
0,STT_GR_0001_0003_28900_to_31600,transcribing,ཨེ་ད་རང་གི་,NaN,https://d38pmlk0v88drf.cloudfront.net/wav16k/S...,2025-06-13 00:49:12.342,NaN,NaN,2.7,STT_GR_0001
1,STT_GR_0001_0021_176000_to_182100,transcribing,གཉིས་ཙུ་བདུན་པ་ཕྱི་དཔག་ཐམས་ཅད་ཚང་ནས། དེ་ནས་གཉེ...,གཉིས་ཙུ་བདུན་པ་ཕྱི་དཔག་ཐམས་ཅད་ཚང་ནས། དེ་ནས་གཉེ...,https://d38pmlk0v88drf.cloudfront.net/wav16k/S...,2025-06-13 00:49:12.342,2025-06-14 20:57:01.034,NaN,6.1,STT_GR_0001
2,STT_GR_0001_0022_182500_to_189800,transcribing,དེ་ནས་སངས་རྒྱས་སེམས་བསྐྱིད་ནས་བཟུང་ཡོད་རེད་དེ།...,NaN,https://d38pmlk0v88drf.cloudfront.net/wav16k/S...,2025-06-13 00:49:12.342,NaN,NaN,7.3,STT_GR_0001
3,STT_GR_0001_0023_192100_to_196100,transcribing,དེ་ནས་ཚང་རྒྱལ་ཡོད་རྒྱོ་དེ་གང་རེད་དེན་ན། དཔེར་ན...,NaN,https://d38pmlk0v88drf.cloudfront.net/wav16k/S...,2025-06-13 00:49:12.342,NaN,NaN,4.0,STT_GR_0001
4,STT_GR_0001_0025_201100_to_204400,transcribing,སང་རྒྱུ་དེ་ག་རེ་བཟང་། རྒྱད་རྒྱུ་དེ་ག་རེ་རྒྱད།,NaN,https://d38pmlk0v88drf.cloudfront.net/wav16k/S...,2025-06-13 00:49:12.342,NaN,NaN,3.3,STT_GR_0001


In [10]:
import pandas as pd

# Read both dataframes


# Create a mapping dictionary from df2
transcript_map = dict(zip(df2['file_name'], df2['transcript']))

# Create new 'uni' column in df1 by mapping values from df2
df['uni'] = df['file_name'].map(transcript_map)

# Save the updated df1
df.to_csv('processed_test1_with_uni.csv', index=False)

print("Number of rows in df1:", len(df))
print("Number of rows in df2:", len(df2))
print("Number of matched transcripts:", df['uni'].notna().sum())
print("\nFirst few rows of updated df1:")
print(df[['file_name', 'uni']].head())

Number of rows in df1: 978
Number of rows in df2: 16431
Number of matched transcripts: 749

First few rows of updated df1:
                           file_name  \
0   STT_GR_0001_0010_93600_to_106200   
1  STT_GR_0033_0105_224320_to_225856   
2    STT_GR_0032_0004_19067_to_24988   
3  STT_GR_0033_0099_212224_to_213024   
4    STT_GR_0033_0025_56251_to_58684   

                                                 uni  
0  ཁོང་དེ་ནས་དགེ་འདུན་ཁྲི་ལ་སྔོན་ཟླ་རེ་བོད་གླེང་ད...  
1                           ཕ་གི་ཕར་རང་རེ་རྫས་ཚར་ལེ།  
2  དབང་པོ་གསལ་མཁན་དེ་ཡིས་དབང་སྐུར་དེ་ལ་གསང་རྫས་མཁ...  
3                                     ཐབས་དང་ཟབ་ཅིང་  
4  ཁོ་ཚོས་གཅིག་གོ་ཐུབ་ཀི་འདུག ཡག་པོ་འདུག་བསམ་གི་འ...  


In [11]:
df = pd.read_csv('processed_test1_with_uni.csv')

In [12]:
len(df)

978

In [14]:
df = df.dropna(subset=['uni'])


In [15]:
len(df)

749